In [1]:
"""Scripts for analyzing of phantom outputs.

This script generate lightcurves (LC) by doing radiative transfer on a grid.

Author: Chunliang Mu (at Macquarie University, expected duration 2023-2026)


-------------------------------------------------------------------------------

Side note: Remember to limit line length to 79 characters according to PEP-8
    https://peps.python.org/pep-0008/#maximum-line-length    
which is the length of below line of '-' characters.

-------------------------------------------------------------------------------

"""

"Scripts for analyzing of phantom outputs.\n\nThis script generate lightcurves (LC) by doing radiative transfer on a grid.\n\nAuthor: Chunliang Mu (at Macquarie University, expected duration 2023-2026)\n\n\n-------------------------------------------------------------------------------\n\nSide note: Remember to limit line length to 79 characters according to PEP-8\n    https://peps.python.org/pep-0008/#maximum-line-length    \nwhich is the length of below line of '-' characters.\n\n-------------------------------------------------------------------------------\n\n"

.

# Def

.

## Imports & Settings

In [2]:
import math
import numpy as np
from numpy import pi
from astropy import units
from astropy import constants as const
import matplotlib.pyplot as plt
import matplotlib as mpl
from numba import jit
import sarracen
import itertools
from scipy import integrate, fft
from scipy.spatial import kdtree
# fix weird moviepy cannot find my ffmpeg exe error
try: from moviepy import editor
except RuntimeError: import os; os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"
#from moviepy.editor import ImageSequenceClip
#from os import path

from datetime import datetime, UTC
now_utc = lambda: datetime.now(UTC)
now = now_utc

# fix numpy v1.* compatibility
try: np.trapezoid
except AttributeError: np.trapezoid = np.trapz

In [3]:
# import my modules listed in ./main/

import clmuphantomlib as mupl
from clmuphantomlib            import MyPhantomDataFrames, get_eos, get_col_kernel_funcs
from clmuphantomlib.log        import is_verbose, say
#from clmuphantomlib.settings   import DEFAULT_SETTINGS as settings
from clmuphantomlib.units_util import get_val_in_unit, set_as_quantity #, get_units_field_name, get_units_cgs
from clmuphantomlib.io         import json_dump, json_load
from clmuphantomlib.eos        import get_eos_opacity
from clmuphantomlib.light      import get_optical_depth_by_ray_tracing_3D, get_photosphere_on_ray
#from clmuphantomlib.sph_interp import get_col_kernel_funcs

from multiprocessing import cpu_count, Pool #Process, Queue
NPROCESSES = 1 if cpu_count() is None else max(cpu_count(), 1)

In [4]:
# settings
#
#   imported from script_input.py file


from script_LCGen__input import (
    verbose, verbose_loop,
    interm_dir, output_dir, JOB_PROFILES_DICT, job_nicknames, xyzs_list, no_xy, no_xy_txt,
    unitsOut, PHOTOSPHERE_TAU, AVG_KC_PP, wavlens, use_Tscales, nsample_pp, z_olim_kc,
)
from _sharedFuncs import mpdf_read

unitsOutTxt = {  key  : unitsOut[key].to_string('latex_inline') for key in unitsOut.keys() }


# set metadata
with open("_metadata__input.json", 'r') as f:
    metadata = json_load(f)
metadata['Title'] = "Getting light curves by intergrating across a grid of rays"
metadata['Description'] = f"""Getting light curves by intergrating across a grid of rays with the same directions
for dump file data generated by phantom
"""


plt.rcParams.update({'font.size': 20})
if __name__ == '__main__' and is_verbose(verbose, 'note'):
    # remember to check if name is '__main__' if you wanna say anything
    #    so when you do multiprocessing the program doesn't freak out
    say('note', "script", verbose, f"Will use {NPROCESSES} processes for parallelization")


*   Note   :    script:
	Will use 8 processes for parallelization


In [5]:
from clmuphantomlib.log import say, is_verbose
from clmuphantomlib.geometry import get_dist2_between_2pt, get_closest_pt_on_line
from clmuphantomlib.sph_interp import get_sph_interp, get_sph_gradient, get_h_from_rho, get_no_neigh, _get_sph_interp_phantom_np
from clmuphantomlib.units_util import set_as_quantity, set_as_quantity_temperature, get_units_field_name
from clmuphantomlib.eos import EoS_Base
#from clmuphantomlib.light import integrate_along_ray_grid, integrate_along_ray_gridxy

#  import (general)
import numpy as np
from numpy import typing as npt
import numba
from numba import jit, prange
import sarracen

from clmuphantomlib.geometry import get_dist2_from_pts_to_line, get_dist2_from_pt_to_line_nb, get_ray_unit_vec, get_rays_unit_vec

## Functions

### LC integration

#### Backup codes

#### Test codes

In [6]:
# test runs
@jit(nopython=True, parallel=True, fastmath=True)
def _integrate_along_rays_gridxy_sub_parallel_olim(
    pts_ordered          : npt.NDArray[np.float64],    # (npart, 3)-shaped
    hs_ordered           : npt.NDArray[np.float64],    # (npart,  )-shaped
    mkappa_div_h2_ordered: npt.NDArray[np.float64],    # (npart,  )-shaped
    srcfuncs_ordered     : npt.NDArray[np.float64],    # (npart,  )-shaped
    rays_xy              : npt.NDArray[np.float64],    # (nray,  2)-shaped
    ray_areas            : npt.NDArray[np.float64],    # (nray,   )-shaped
    kernel_rad           : float,
    kernel_col           : numba.core.registry.CPUDispatcher,
    kernel_csz           : numba.core.registry.CPUDispatcher,
    kernel_w             : numba.core.registry.CPUDispatcher,
    pts_order            : npt.NDArray[np.float64],    # (npart,  )-shaped
    rel_tol              : float = 1e-16, # because float64 has only 16 digits accuracy
    nsample_pp           : int   = 100,  # no of sample points for integration
    z_olim_kc            : float = 1.152,  # col kernel limit for when srcfunc began to count
    photosphere_tau      : float = 2./3.,
) -> tuple[
    npt.NDArray[np.float64],    # rads
    npt.NDArray[np.float64],    # rads_olim
    npt.NDArray[np.float64],    # phkcs
    npt.NDArray[np.float64],    # pphzs
    npt.NDArray[np.int64  ],    # indes
    npt.NDArray[np.float64],    # contr
    npt.NDArray[np.float64],    # jfact
    npt.NDArray[np.float64],    # jfact_olim
    npt.NDArray[np.float64],    # estis
]:
    """Sub process for integrate_along_ray_gridxy(). Numba parallel version (using prange).

    Note: in this version,
    the source function
        outside of the outermost particle (within one h of the ray)'s z
        are ignored.
    
    ---------------------------------------------------------------------------

    Calculating the luminosity using
    $$
    L \approx
        4 \pi \sum_i \sum_j \triangle A_i
            S_j \frac{\kappa_j m_j}{h_j^2}
            \int
                e^{
                    -\sum_k \frac{\kappa_k m_k}{h_k^2}
                    w_\mathrm{csz}(q_{xy, ik}, \, -q_{z, k})
                }
                w(q_{ij}) d(q_{z, j})
    $$

    Unit vec must be [0., 0., 1.] (i.e. all rays must point upwards towards +z)

    Private function. Assumes specific input type. See source code comments.

    Returns
    -------
    rads, rads_olim, phkcs, pphzs, indes, contr, jfact, jfact_olim, estis
    
    rads: (nray,)-shaped np.ndarray[float]
        Radiance (i.e. specific intensities) for each ray.

    rads_olim: (nray,)-shaped np.ndarray[float]
        Radiance (i.e. specific intensities) for each ray, excluding unresolved surface.

    phkcs: (nray,)-shaped np.ndarray[float]
        Summed column kernel values at the photosphere.
        summed of all particle's columne kernel if photosphere not found

    pphzs: (nray,)-shaped np.ndarray[float]
        locations of the photosphere (i.e. z value).
        np.nan if photosphere not found

    indes: (nray,)-shaped np.ndarray[int]
        indexes of the max contribution particle

    contr: (nray,)-shaped np.ndarray[float]
        relative contribution (in fractions) of the max contribution particle

    jfact: (npart,)-shaped np.ndarray[float]
        Contribution factor for j-th particle.
        Cannot be used as a way to calc lum
            because we are additionally assuming the outermost particles has srcfunc of zero.

    jfact_olim: (npart,)-shaped np.ndarray[float]
        Contribution factor for j-th particle.
        Multiply it with 4 * pi * srcfuncs and sum it up as an alternative way to get the luminosity.
        Will be zero if particle is not used.

    estis: (nray,)-shaped np.ndarray[float]
        Estimations of radiance (i.e. specific intensities) for each ray, using old method
        (Also estis means 'was' in Esperanto, so it's a fitting name)

    """
    #raise NotImplementedError

    nray  = len(rays_xy)
    npart = len(srcfuncs_ordered)
    ndim  = pts_ordered.shape[-1]
    rads  = np.zeros(nray)
    rads_olim = np.zeros(nray)
    indes = np.zeros(nray, dtype=np.int64)    # indexes of max contribution particle
    contr = np.zeros(nray)     # relative contribution of the max contribution particle
    # ifact = np.zeros(nray)     # effective xsec for i-th ray  # NOTE: ifact = pones * ray_areas
    jfact = np.zeros(npart)    # effective xsec for j-th particle
    jfact_olim = np.zeros(npart)    # effective xsec for j-th particle
    phkcs = np.zeros(nray)    # photosphere column kernels
    pphzs = np.full(nray, np.nan)    # photosphere z
    # pones = np.zeros(nray)
    # ptaus = np.full(nray, np.nan)    # lower bound of the optical depth
    estis = np.zeros(nray)
    
    # error tolerance of tau (part 1)
    # #    this is aonly here for reference- value will be updated later
    # tol_tau_base = np.log(srcfuncs_ordered.sum()) - np.log(rel_tol)


    # cache calcs
    hrs_ordered = hs_ordered * kernel_rad
    # #   limits (approx)
    # z_min = pts_ordered[-1, 2] - hrs_ordered[-1]
    # z_max = pts_ordered[ 0, 2] + hrs_ordered[ 0]

    # loop over ray
    for i in prange(nray):
        ray_xy  = rays_xy[i]
        ray_area= ray_areas[i]
        rad     = 0.
        rad_olim= 0.
        rad_est = 0.   # estimation of radiance (i.e. rad)
        # dans_max_tmp = 0.
        dfac_max_tmp = 0.
        ind     = -1
        fac     = 0. # effectively <1>
        dfac    = 0. # factor
        used_j  = np.int64(0)    # j = used_indexes[used_j]
        ji      = np.int64(0)
        ji_r    = np.int64(0)
        z_olim = 0.

        #   xy-grid specific solution
        ray_x = ray_xy[0]
        ray_y = ray_xy[1]

        

        # First, try to find out how many relevant particles are there

        nused_i = 0    # no of used particles for i-th ray
        srcfuncs_tot_relevant = 0.
        for j in range(npart):
            x_j = pts_ordered[j, 0]
            y_j = pts_ordered[j, 1]
            hr = hrs_ordered[j]
            # check if the particle is within range
            #   xy-grid specific solution
            if ray_x - hr < x_j and x_j < ray_x + hr and ray_y - hr < y_j and y_j < ray_y + hr:
                nused_i += 1
                srcfuncs_tot_relevant += srcfuncs_ordered[j]

        
        if nused_i <= 0:
            continue    # skip

        
        # Then, get the indexes etc info of these relevant particles

        
        used_indexes = np.full(nused_i, -1, dtype=np.int64)
        used_dtaus   = np.full(nused_i, np.nan)
        used_qs_xy   = np.full(nused_i, np.nan)
        used_dzs     = np.full(nused_i, np.nan)
        tol_tau_base_i = np.log(srcfuncs_tot_relevant) - np.log(rel_tol)    # error tolerance of tau (part 1)

        used_j = 0    # j = used_indexes[used_j]
        tau    = 0.
        rad_est= 0.   # estimation of radiance (i.e. rad)
        kc     = 0.
        kcs    = 0.   # col kernel cummulative sum
        for j in range(npart):
            x_j = pts_ordered[j, 0]
            y_j = pts_ordered[j, 1]
            hr  = hrs_ordered[j]
            
            # check if the particle is within range
            #   xy-grid specific solution
            if ray_x - hr < x_j and x_j < ray_x + hr and ray_y - hr < y_j and y_j < ray_y + hr:
                r2_xy = (x_j - ray_x)**2 + (y_j - ray_y)**2
                dz2 = hr**2 - r2_xy
                if dz2 > 0:
                    z_j = pts_ordered[j, 2]
                    h = hs_ordered[ j]
                    q_xy = np.sqrt(r2_xy)/h

                    # log
                    dz = np.sqrt(dz2)
                    mkappa_div_h2 = mkappa_div_h2_ordered[j]
                    srcfunc = srcfuncs_ordered[j]
                    kc = kernel_col(q_xy, ndim)
                    dtau = mkappa_div_h2 * kc
                    kcs += kc
                            
                    used_indexes[used_j] = j
                    used_dtaus[  used_j] = dtau
                    used_qs_xy[  used_j] = q_xy
                    used_dzs[    used_j] = dz
                    used_j += 1

                    dfac = np.exp(-tau) * (1. - np.exp(-dtau))
                    rad_est += dfac * srcfunc
                    tau += dtau
                    
                    # terminate the calc for this ray if tau is sufficient large
                    #    such that the relative error on rad is smaller than rel_tol
                    # i.e. since when tau > np.log(srcfuncs_ordered.sum()) - np.log(rel_tol) - np.log(rad),
                    #    we know that rad[i] - rad[i][k] < rel_tol * rad[i]
                    # see my notes for derivation
                    #    This should mean tau > photosphere_tau, but let's check it anyway to make sure
                    if tau > tol_tau_base_i - np.log(rad_est) and tau > photosphere_tau:
                        break
        else:
            # ptaus[i] = tau
            phkcs[i] = kcs    # photosphere not found if didn't break
        nused_i = used_j     # update used indexes size
        used_dtaus = used_dtaus[:nused_i]
        estis[i] = rad_est
        
        
        if nused_i <= 0:
            continue    # skip

        

        # Now, do radiative transfer

        z_olim_not_found = True
        photosphere_not_found = True
        kcs_j  = np.zeros(nsample_pp)
        for used_j in range(nused_i):
            j    = used_indexes[used_j]
            q_xy = used_qs_xy[used_j]
            q2_xy= q_xy**2
            h    = hs_ordered[ j]
            hr   = h * kernel_rad
            mkappa_div_h2 = mkappa_div_h2_ordered[j]
            srcfunc = srcfuncs_ordered[j]
            
            z       = pts_ordered[j, 2]
            dz      = used_dzs[used_j]
            zmdz    = z - dz
            zpdz    = z + dz
            dzs_ji  = (zpdz - zmdz) / nsample_pp
            dqz_j   = dzs_ji / h
            # note zs_j is from away from observer to closer to observer
            zs_j    = np.linspace(zmdz + dzs_ji/2., zpdz - dzs_ji/2., nsample_pp)
            taus_j  = np.zeros(nsample_pp)
            # used_pr = 0    # index of the particle index that is 'fully' ahead of j-th particle
            tau_pr  = 0.   # \tau'_j - the summed optical depth for particles that are 'fully' ahead of j-th particle
            
            dfac  = 0.
            dfac_olim = 0.    # same as dfac, except that discounts the part outside z_olim (outmost particle loc) to zero
            ddfac = 0.    # temp storage
            if z_olim_not_found:
                kcs_j[:] = 0
            # get optical depth
            #assert np.isclose(dzs_ji, zs_j[1] - zs_j[0])
            for used_k in range(nused_i):
                k = used_indexes[used_k]
                z_k  = pts_ordered[k, 2]
                dz_k = used_dzs[used_k]
                if zmdz - dz_k < z_k:
                    # particle being relevant
                    if           z_k < zpdz + dz_k:
                        # particle range intersects
                        h_k = hs_ordered[k]
                        mkappa_div_h2_k = mkappa_div_h2_ordered[k]
                        q_xy_k = used_qs_xy[used_k] #((x_k - ray_x)**2 + (y_k - ray_y)**2)**0.5 / h_k
                        for ji in range(nsample_pp):
                            q_z_k  = (zs_j[ji] - z_k) / h_k
                            kc     = kernel_csz(q_xy_k, -q_z_k, ndim)
                            taus_j[ji] += mkappa_div_h2_k * kc
                            if z_olim_not_found:
                                kcs_j[ji] += kc
                    else:
                        # particle is fully ahead
                        tau_pr += used_dtaus[used_k]
                        if z_olim_not_found:
                            q_xy_k = used_qs_xy[used_k]
                            kcs_j += kernel_col(q_xy_k, ndim)
            # find z_olim
            if z_olim_not_found:
                if kcs_j[0] > z_olim_kc:    # found
                    for ji_r in range(nsample_pp):
                        ji = nsample_pp - ji_r - 1
                        if kcs_j[ji] > z_olim_kc:
                            z_olim = zs_j[ji]
                            z_olim_not_found = False
                            break

            # find photosphere
            if photosphere_not_found:
                if taus_j[0] > photosphere_tau:    # found
                    for ji_r in range(nsample_pp):
                        ji = nsample_pp - ji_r - 1
                        if taus_j[ji] > photosphere_tau:
                            pphzs[i] = zs_j[ji]
                            phkcs[i] = kcs_j[ji]
                            photosphere_not_found = False
                            break
            
            # integrate through opitcal depth for j
            for ji in range(nsample_pp):
                # ****** Integration pending improvement ******
                # ****** same for the integration in w_csz ******
                q_ji = np.sqrt(q2_xy + ((zs_j[ji] - z)/h)**2)
                ddfac = np.exp(-taus_j[ji]) * kernel_w(q_ji, ndim) * dqz_j
                dfac += ddfac
                if not z_olim_not_found and zs_j[ji] < z_olim:
                    dfac_olim += ddfac

            tmp = mkappa_div_h2 * np.exp(-tau_pr)
            dfac *= tmp
            dfac_olim *= tmp

            # get radiance contribution
            fac      += dfac                # for getting <1>
            rad      += dfac      * srcfunc # for getting <S>
            rad_olim += dfac_olim * srcfunc
            jfact[      j] += dfac      * ray_area
            jfact_olim[j] += dfac_olim * ray_area
            # note down the largest contributor
            if dfac > dfac_max_tmp:
                dfac_max_tmp = dfac
                ind = pts_order[j]
        
        rads[ i] = rad
        rads_olim[i] = rad_olim
        indes[i] = ind
        if rad > 0: contr[i] = dfac_max_tmp / fac  # dans_max_tmp / rad
        # pones[i] = fac
    
    return rads, rads_olim, phkcs, pphzs, indes, contr, jfact, jfact_olim, estis


In [7]:
# integrate with error estiamtes

def integrate_along_rays_gridxy(
    sdf         : sarracen.SarracenDataFrame,
    srcfuncs    : npt.NDArray[np.float64],    # (npart,  )-shaped
    srcfuncs_err: None|npt.NDArray[np.float64],    # (npart,  )-shaped
    rays        : npt.NDArray[np.float64],    # (nray, 2, 3)-shaped
    ray_areas   : npt.NDArray[np.float64],    # (nray,     )-shaped
    ray_unit_vec: None|npt.NDArray[np.float64] = None,    # (nray, 3)-shaped
    kernel      : None|sarracen.kernels.BaseKernel = None,
    kernel_col  : None|numba.core.registry.CPUDispatcher = None,
    kernel_csz  : None|numba.core.registry.CPUDispatcher = None,
    hfact       : None|float = None,
    parallel    : bool = False,
    err_h       : float = 1.0,
    rel_tol     : float = 1e-16,
    nsample_pp  : int   = 1000,
    z_olim_kc   : float = 1.152,
    photosphere_tau: float = 2./3.,
    sdf_kdtree  : None|kdtree.KDTree = None,
    xyzs_names_list : list = ['x', 'y', 'z'],
    verbose     : int = 3,
):
    """Backward integration of source functions along a grided ray (traced backwards), weighted by optical depth.
    ---------------------------------------------------------------------------
    
    Assuming all rays facing +z direction
    (with the same ray_unit_vec [0., 0., 1.])

    WARNING: will overwrite sdf['srcfunc'] if srcfuncs_err is None.
    
    
    Parameters
    ----------
    sdf: sarracen.SarracenDataFrame
        Must contain columns: x, y, z, h, m, kappa
        
    rays: (nray, 2, 3)-shaped array
        Representing the ray trajectory. Currently only straight infinite lines are supported.
        each ray is of the format:
        [[begin point], [end point]]
        where the end point is closer to the observer.

    srcfuncs: 1D array
        arrays describing the source function for every particle
        
    kernel: sarracen.kernels.base_kernel
        Smoothing kernel for SPH data interpolation.
        If None, will use the one in sdf.

    kernel_csz: func
        Cumulative summed kernel along z axis:
        $ \int_{-w_\mathrm{rad}}^{q_z} w(\sqrt{q_{xy}^2 + q_z^2}) dq_z $

    hfact : None|float

    parallel: bool
        If to use the numba parallel function

    err_h: float ( > 0. )
        determine confidence level.
        e.g.,
            1.0 will give error assuming error range is +/-1.0 smoothing length h;
            0.5 will give error assuming error range is +/-0.5 smoothing length h;
            etc. etc.
            
    rel_tol : float
        maximum relative error tolerence per ray.
        Default 1e-15 because float64 is only accurate to ~16th digits.
        
    nsample_pp : int
        no of sample points per particle for integration

    z_olim_kc : float
        col kernel limit for when srcfunc began to count
        1.152 because it is where there's an average of 14.5 particles (1/4 N_neigh) on the outer side, as
            quad(lambda q_xy: N_neigh/4*w_col(q_xy,3)*q_xy, 0., 2.,) gives 2.304

    sdf_kdtree : kdtree.KDTree
        KDTree built from sdf[['x', 'y', 'z']], for fast neighbour search.
        if None, will build one.
        
    xyzs_names_list: list
        list of names of the columns that represents x, y, z axes (i.e. coord axes names)
        MUST INCLUDE ALL THREE AXES LABELS.
        If only 2 is included, WILL ASSUME IT IS 2D CACULATIONS.
    
    Returns
    -------
    lum, lum_err, rads, jfact_used,
    lum_olim, lum_err_olim, rads_olim, jfact_olim_used,
    estis, phkcs, pphzs, indes, contr, pts_order_used
    
    rads: np.ndarray
        Radiance (i.e. specific intensities) for each ray.
    
    """


    # init
    npart : int = len(sdf)
    nray  : int = len(rays)
    if kernel is None: kernel = sdf.kernel
    kernel_rad = float(kernel.get_radius())
    #kernel_col = kernel.get_column_kernel_func(samples=1000) # w integrated from z
    if kernel_col is None or kernel_csz is None:
        kernel_col, kernel_csz, _, _ = get_col_kernel_funcs(kernel)
    if ray_unit_vec is None: ray_unit_vec = get_ray_unit_vec(rays[0])
    
    pts    = np.array(sdf[xyzs_names_list], order='C')    # (npart, 3)-shaped array (must be this shape for pts_order sorting below)
    hs     = np.array(sdf[ 'h'           ], order='C')    # npart-shaped array
    masses = np.array(sdf[ 'm'           ], order='C')
    kappas = np.array(sdf[ 'kappa'       ], order='C')
    srcfuncs = np.array(srcfuncs          , order='C')
    ndim   = pts.shape[-1]
    mkappa_div_h2_arr = masses * kappas / hs**(ndim-1)
    
    # sanity check
    if is_verbose(verbose, 'err') and not np.allclose(ray_unit_vec, get_rays_unit_vec(rays)):
        raise ValueError(f"Inconsistent ray_unit_vec {ray_unit_vec} with the rays.")

    if is_verbose(verbose, 'warn') and ndim != 3:
        say('warn', None, verbose, f"ndim == {ndim} is not 3.")

    if is_verbose(verbose, 'fatal') and not np.allclose(ray_unit_vec, np.array([0., 0., 1.])):
        raise NotImplementedError(
            f"Unsupported {ray_unit_vec=}:"+
            "currently all rays must point towards +z direction (ray_unit_vec = np.array([0., 0., 1.])) ")
    # *** warning: the following line only works with +z point rays ***
    rays_xy = rays[:, 0, 0:2]

    # (npart-shaped array of the indices of the particles from closest to the observer to the furthest)
    pts_order             = np.argsort( np.sum(pts * ray_unit_vec, axis=-1) )[::-1]
    pts_ordered           = pts[     pts_order]
    hs_ordered            = hs[      pts_order]
    mkappa_div_h2_ordered = mkappa_div_h2_arr[pts_order]
    srcfuncs_ordered      = srcfuncs[pts_order]

    # get used particles indexes
    if parallel:
        rads, rads_olim, phkcs, pphzs, indes, contr, jfact, jfact_olim, estis = _integrate_along_rays_gridxy_sub_parallel_olim(
            pts_ordered, hs_ordered, mkappa_div_h2_ordered, srcfuncs_ordered,
            rays_xy, ray_areas, kernel_rad, kernel_col, kernel_csz, kernel.w, pts_order,
            rel_tol=rel_tol, nsample_pp=nsample_pp, z_olim_kc=z_olim_kc, photosphere_tau=photosphere_tau)
    else:
        raise NotImplementedError("parallel=False version of this function not yet implemented.")

    jused = np.where(jfact)
    jfact_used = jfact[jused]
    jfact_olim_used = jfact_olim[jused]
    pts_order_used = pts_order[jused]
    
    lum  = 4 * pi * (rads * ray_areas).sum()
    lum2 = 4 * pi * (srcfuncs_ordered[jused] * jfact_used).sum()
    lum_olim  = 4 * pi * (rads_olim * ray_areas).sum()
    lum_olim2 = 4 * pi * (srcfuncs_ordered[jused] * jfact_olim_used).sum()
    say('debug', None, verbose,
        f"{lum = }, {lum2 = }    (code unit)",
        f"{lum_olim = }, {lum_olim2 = }    (code unit)",
        f"{rads.shape=}, {ray_areas.shape=}",
        f"{srcfuncs[jused].shape=}, {jfact_used.shape=}",
    )
    #assert np.isclose(lum, lum2)

    if srcfuncs_err is None:
        # calc error of source function now
        sdf['srcfunc'] = srcfuncs
        srcfuncs_grad_used, nneighs_z_used = get_sph_gradient(
            sdf,
            val_names   ='srcfunc',
            locs        = pts_ordered[     jused],
            vals_at_locs= srcfuncs_ordered[jused],
            hs_at_locs  = hs_ordered[      jused],
            kernel      = kernel,
            hfact       = hfact,
            sdf_kdtree  = sdf_kdtree,
            ndim        = ndim,
            xyzs_names_list=xyzs_names_list,
            parallel    = parallel,
            return_nneighs_z = True,
            verbose     = verbose,
        )
        srcfuncs_grad_used = srcfuncs_grad_used[:, :, 0]    # get_sph_gradient returns a (nlocs, ndim, nvals)-shaped np.ndarray
        # srcfuncs_err_used: (nlocs,)-shaped np.ndarray
        srcfuncs_err_used = np.sum(srcfuncs_grad_used**2, axis=1)**0.5 * hs_ordered[jused] * err_h
        # # put a floor on error range if particle position is poorly resolved
        # #    pres_used = poorly_resolved_used
        # #    *** this bit only works if ray is pointing towards +z ***
        # pres_used = nneighs_z_used[:, 0] < 8
        # srcfuncs_err_used = np.where(
        #     np.logical_and(pres_used, srcfuncs_ordered[jused] > srcfuncs_err_used),
        #     srcfuncs_ordered[jused],    # no need for abs since srcfuncs are already positive
        #     srcfuncs_err_used,
        # )
    else:
        srcfuncs_err_used = srcfuncs_err[pts_order_used]
        # pres_used = np.zeros(len(jused), dtype=bool)

    lum_err = 4 * pi * (((srcfuncs_err_used * jfact_used)**2).sum())**0.5
    lum_err_olim = 4 * pi * (
        ((srcfuncs_err_used * jfact_olim_used)**2).sum()    # uncertainty from normal inside particles
        + (srcfuncs_ordered[jused] * (jfact_used - jfact_olim_used)).sum()**2    # uncertainty from outermost particles
    )**0.5

    # number of particles before photosphere (based on column kernel) per pixel
    pphns = phkcs / AVG_KC_PP

    if is_verbose(verbose, 'info'):
        nused = len(jfact_used)
        say('info', None, verbose,
            f"{nused} particles actually participated calculation",
            f"({int(nused/npart*10000)/100.}% of all particles,",
            f"average {int(nused/nray*100)/100.} per ray.)\n",
            f"Photosphere resolution {np.average(pphns) = }, {np.std(pphns) = }\n",
            f"overall resolution {jfact_used.sum() / np.max(jfact_used) = :.1f}\n" if nused else "",
            f"overall resolution (olim) {jfact_olim_used.sum() / np.max(jfact_olim_used) = :.1f}\n" if nused else "",
            # f"Among which, {np.count_nonzero(pres_used)} are poorly resolved (less than 8 neighbours with higher z))\n",
            sep=' ')

    
    return (
        lum, lum_err, rads, jfact_used,
        lum_olim, lum_err_olim, rads_olim, jfact_olim_used,
        estis, pphns, pphzs, indes, contr, pts_order_used
    )


### rays grid generation

In [8]:
def get_xy_grids_of_rays(
    sdf  : None|sarracen.SarracenDataFrame = None,
    #dXs  : None|list[list[float], list[float]]|np.ndarray= None,
    no_xy: tuple[int, int] = (32, 32),
    #orig_vec: np.ndarray = np.zeros(3),
    frac_contained: float = 100., #99.73,
    use_adaptive_grid: bool = False,
    xyzs_names_list : list = ['x', 'y', 'z'],
    w_rad: None|float = None,
    verbose: int = 3,
) -> tuple[np.ndarray, np.ndarray, list[np.ndarray]]:
    """Get a grid of rays (must pointing at z direction (i.e. xyzs_names_list[-1] direction) for now).

    Supply either sdf or both dx and dy.
    
    Parameters
    ----------
    sdf: sarracen.SarracenDataFrame

    no_xy: tuple[int, int]
        number of the rays per axes.
        
    frac_contained : float
        Suggested percentage of the particle that are contained within the grid. in (0, 100]

    use_adaptive_grid : bool
        if True,
            will scale dXs according to particle distribution instead of even intervals,
            if dXs is None or (None, None).

    xyzs_names_list: list
        list of names of the columns that represents x, y, z axes (i.e. coord axes names)
        WARNING: since ndim==len(xyzs_names_list), if len(xyzs_names_list) !=3 will resulting non-3D results.
            In which case you will need to change no_xy as well.


    Returns: rays, areas, dXs
    -------
    rays: (no_ray, 2, 3)-shaped np.ndarray

    areas: (no_ray)-shaped np.ndarray
        areas corresponding to each ray in the grid
        
    dXs: list of no_xy[i]-shaped np.ndarray
        width of the grid cells. in sdf units['dist'].
        
    """

    unit_vec = np.zeros(len(xyzs_names_list))
    unit_vec[-1] = 1.
    #x0, y0, z0 = orig_vec
    z0 = 0.    # z value for rays
    if w_rad is None:
        w_rad = sdf.kernel.get_radius()

    # sanity checks
    if is_verbose(verbose, 'warn') and len(xyzs_names_list) != 3:
        say('warn', 'get_xy_grids_of_rays()', verbose,
            f"xyzs_names_list being {xyzs_names_list}, its len = {len(xyzs_names_list)} is not 3.",
            f"This means we are assuming {len(xyzs_names_list)}D.")
    if is_verbose(verbose, 'error') and len(xyzs_names_list) != len(no_xy) + 1:
        say('error', 'get_xy_grids_of_rays()', verbose,
            f"ndim (=={len(xyzs_names_list)}) != len(no_xy) (=={len(no_xy)}) + 1",
            f"i.e. asked ray grid dimension {no_xy} does not makes sense.",
            "This will likely cause error in the next steps.")
    
    # get dx & dy
    frac_contained_m = 50. - frac_contained / 2.
    frac_contained_p = 50. + frac_contained / 2.

    Xs_edges = []
    for i, label in enumerate(xyzs_names_list[:-1]):
        #i0 = orig_vec[i]
        if use_adaptive_grid:
            # fraction points for the adaptive grid
            fracs = np.linspace(frac_contained_m, frac_contained_p, no_xy[i]+1)
            # edge points for the grid
            Xs_edges.append(
                np.percentile(np.concatenate((sdf[label] - w_rad*sdf['h'], sdf[label] + w_rad*sdf['h'])), fracs))
        else:
            Xs_edges.append(
                np.linspace(
                    *np.percentile(
                        np.concatenate(
                            (sdf[label] - w_rad*sdf['h'], sdf[label] + w_rad*sdf['h']),
                        ), (frac_contained_m, frac_contained_p),
                    ), no_xy[i]+1,
                )
            )

    dXs = np.array([np.diff(Xi_edges) for Xi_edges in Xs_edges])    # each item is (no_xy[i]+1,)-shaped
    Xs_centers = np.array([Xi_edges[:-1] + dXi/2. for dXi, Xi_edges in zip(dXs, Xs_edges)])    # each item is (no_xy[i],)-shaped

    # Note: orig_vecs must be 2D (i.e. in shape of (no_ray, 3))
    orig_vecs = np.array([[*xy, z0] for xy in itertools.product(*Xs_centers)])
    #orig_vecs = [[[x, y, z0] for x, y in zip(xs, ys)] for xs, ys in zip(*np.meshgrid(*xys))]
    areas = np.array([dx*dy for dy in dXs[1] for dx in dXs[0]])

    rays = mupl.geometry.get_rays(orig_vecs=orig_vecs, unit_vecs=unit_vec)
    
    return rays, areas, dXs

### Plotting

In [9]:
def plot_imshow(
    no_xy: tuple[int, int],
    rays: units.Quantity|np.ndarray,
    data: units.Quantity|np.ndarray,
    job_profile  : dict= None,
    file_index   : int = -1,
    title_suffix : str ="",
    notes        : dict= None,
    data_label   : str ="",
    save_label   : str ="",
    xyzs         : str|list[str] = 'xyz',
    out_exts     : list[str] = ['pdf', 'png'],
    norm=None,
    cmap=None,
    output_dir:str|None=None,
    verbose = 4,
):
    """Plotting a heatmap (contourf) of 1D data located at rays"""


    if not isinstance(data, units.Quantity):
        data = set_as_quantity(data, units.dimensionless_unscaled)

    if not isinstance(rays, units.Quantity):
        rays = set_as_quantity(rays, units.dimensionless_unscaled)

    if job_profile is None:
        job_profile = {
            'plot_title_suffix': '',
            'nickname'         : '',
        }

    #Xs = rays[:, 0, 0]
    #Ys = rays[:, 0, 1]
    rays_val = rays.reshape(*no_xy, *rays.shape[1:]).value
    extent = (
        rays_val[ 0, 0, 0, 0] - (rays_val[ 1, 0, 0, 0] - rays_val[ 0, 0, 0, 0])/2,
        rays_val[-1,-1, 0, 0] + (rays_val[-1,-1, 0, 0] - rays_val[-2,-1, 0, 0])/2,
        rays_val[ 0, 0, 0, 1] - (rays_val[ 0, 1, 0, 1] - rays_val[ 0, 0, 0, 1])/2,
        rays_val[-1,-1, 0, 1] + (rays_val[-1,-1, 0, 1] - rays_val[-1,-2, 0, 1])/2,
    )

    fig, ax = plt.subplots(figsize=(10, 8))
    
    
    cax = ax.imshow(data.reshape(no_xy).T.value, norm=norm, cmap=cmap, origin='lower', extent=extent)
    #cax = ax.contourf(Xs.reshape(no_xy), Ys.reshape(no_xy), data.reshape(no_xy), cmap=cmap)
    fig.colorbar(cax, label=f"{data_label} / {data.unit.to_string('latex_inline')}")
    ax.set_xlabel(f"${xyzs[0]}$ / {rays.unit.to_string('latex_inline')}")
    ax.set_ylabel(f"${xyzs[1]}$ / {rays.unit.to_string('latex_inline')}")
    if notes is not None:
        ax.text(
            0.98, 0.98,
            f"Time = {notes['time']:.1f}\n" + \
            f" $L$ = {notes['lum']:.0f}",
            #f" $L = {notes['lum'].value:.0f}$ {notes['lum'].unit.to_string('latex_inline')}",
            #color = "black",
            ha = 'right', va = 'top',
            transform=ax.transAxes,
        )

    
    if output_dir is not None:
        no_xy_txt = 'x'.join([f'{i}' for i in no_xy])
        outfilename_noext = f"{output_dir}heat_{job_profile['nickname']}_{file_index:05d}_{''.join(xyzs)}_{save_label}_{no_xy_txt}"
        outfilenames = []
    
        # write pdf
        for out_ext in out_exts:
            outfilename = f"{outfilename_noext}.{out_ext}"
            if out_ext == 'pdf':
                ax.set_title('')
            else:
                ax.set_title(f"Heatmap of {data_label}\n{job_profile['plot_title_suffix']}")
            fig.savefig(outfilename)
            outfilenames.append(outfilename)
            if is_verbose(verbose, 'note'):
                say('note', None, verbose, f"Fig saved to {outfilename}.")
        
    return fig, ax, outfilenames

## example
#fig, ax = plot_imshow(
#    no_xy, rays * units.Rsun, anses, data_label="$I$", save_label="I_xyz",
#    job_profile=job_profile, file_index=file_index, output_dir=output_dir)

### Error estimation

In [10]:
def get_sph_neighbours(
    sdf_kdtree : kdtree.KDTree,
    xyz_i      : np.ndarray,
    h_i        : float,
    w_rad      : float,
) -> tuple[np.ndarray, np.ndarray]:
    """Find neighbours of xyz_i within (w_rad*h_i) distance, using k-d tree.
    
    Parameters
    ----------
    sdf_kdtree : kdtree.KDTree
        KDTree built from sdf[['x', 'y', 'z']], for fast neighbour search.
    xyz_i : (ndim,)-shaped numpy array
        position of the querying point
    h_i : float
        Smoothing length
    w_rad: float
        radius of the smoothing kernel w.
    
    Returns: dists, indices
    -------
    dists : np.ndarray
        distances of the neighbouring points to the querying point
    indices : np.ndarray
        indices of the neighbouring points
    """
    npart = sdf_kdtree.n
    dists, indices = sdf_kdtree.query(xyz_i, k=npart, distance_upper_bound=w_rad*h_i)
    indices_indices = indices<npart
    return dists[indices_indices], indices[indices_indices]

In [11]:
# sph error estimation

# found this func in my old codes ../photosphere/Analysis_PhLoc.ipynb

def get_sph_error(
    sdf        : sarracen.SarracenDataFrame,
    target_labels   : str|list[str],
    target_indicies : int|list[int]|np.ndarray = [],
    err_h      : float    = 1.0,
    sdf_kdtree : kdtree.KDTree = None,
    kernel     : sarracen.kernels.BaseKernel = None,
    xyzs_names_list : list = ['x', 'y', 'z'],
    verbose: int = 3,
) -> np.ndarray:    # (ntarget, nval)-shaped
    """Calculate error bar for sarracen data frame.
    
    Assuming 3D.
    
    Parameters
    ----------
    sdf: sarracen.SarracenDataFrame
        Need to contain columns: x, y, z, m, h, rho.
        If density (rho) is not in sdf, will compute rho.
        
    target_labels: str or list of str (len>2)
        Column label of the target data in sdf for error computing
        
    target_indicies: int or list of int or np.ndarray
        indices for particles in sdf for error calculating
        
    err_h: float ( > 0. )
        determine confidence level.
        e.g.,
            1.0 will give error assuming error range is +/-1.0 smoothing length h;
            0.5 will give error assuming error range is +/-0.5 smoothing length h;
            etc. etc.
            
    sdf_kdtree: kdtree.KDTree
        KDTree built from sdf[['x', 'y', 'z']], for fast neighbour search.
        If None, will build one.
        
    kernel: sarracen.kernels.base_kernel
        Smoothing kernel for SPH data interpolation.
        If None, will use the one in sdf.

    xyzs_names_list: list
        list of names of the columns that represents x, y, z axes (i.e. coord axes names)
        MUST INCLUDE ALL THREE AXES LABELS.
        If only 2 is included, WILL ASSUME IT IS 2D CACULATIONS.
    
            
    Returns: dvals
    -------
    dvalsp: (ntarget, nval)-shaped ndarray
        error.
    """
    
    
    # init
    
    xyzs = sdf[xyzs_names_list].to_numpy()
    ms   = sdf['m'   ].to_numpy()
    hs   = sdf['h'   ].to_numpy()
    rhos = sdf['rho' ].to_numpy()
    target_indicies = np.atleast_1d(target_indicies)
    dxes = err_h * hs[target_indicies]
    # assuming 3D in the following calc
    locs = xyzs[target_indicies]
    vals =  sdf[ target_labels ].to_numpy()[target_indicies]
    if vals.ndim == 2: nval = vals.shape[1]
    else:              nval = 1
    ntarget = len(target_indicies)
    ndim = len(xyzs_names_list)
    
    if sdf_kdtree is None:
        sdf_kdtree = kdtree.KDTree(xyzs)
    if kernel is None:
        kernel = sdf.kernel
    kernel_rad = float(kernel.get_radius())
    kernel_w   = kernel.w
        
    neigh_rad = kernel_rad + err_h
    
    # ans array
    dvals = np.full((ntarget, nval), np.nan)
    
    
    for i in range(ntarget):
        loc = locs[i]
        val = vals[i]
        dx  = dxes[i]
        h   = hs[target_indicies[i]]
        # find all neighbours within 3h, this includes all points needed for the calc of error for this particle
        _, neigh_inds = get_sph_neighbours(sdf_kdtree, loc, h, neigh_rad)

        # prepare data
        sdf_temp = sdf.iloc[neigh_inds]
        
        loc_plus_dx = [loc for i in range(ndim*2)]
        for j in range(ndim):
            loc_plus_dx[j][j] += dx
            loc_plus_dx[ndim+j][j] -= dx

        dval_xyz = get_sph_interp(sdf_temp, target_labels, loc_plus_dx, kernel=kernel, verbose=0) - val
        dvals[i] = ((dval_xyz**2).sum(axis=0)/len(dval_xyz))**0.5
    
    #dvals = dvals.squeeze()
    return dvals

### Spectrum Generation (Gray Opacity)

In [12]:
# spectrum generation


def B_vu(freqs: units.Quantity, T: units.Quantity) -> units.Quantity:
    return 2 * const.h / const.c**2 * freqs**3 / (np.exp(const.h * freqs / (const.k_B * T)) - 1)

def B_wav(wavlens: units.Quantity, T: units.Quantity) -> units.Quantity:
    return 2 * const.h * const.c**2 / wavlens**5 / (np.exp(const.h * const.c / (wavlens * const.k_B * T)) - 1)



CONST_H = const.h.cgs.value
CONST_C = const.c.cgs.value
CONST_K_B = const.k_B.cgs.value
CONST_SIG = const.sigma_sb.cgs.value

@jit(nopython=True, fastmath=True)
def B_vu_nb(freqs_Hz: npt.NDArray[float], T_K: float) -> npt.NDArray[float]:
    return 2 * CONST_H / CONST_C**2 * freqs**3 / (np.exp(CONST_H * freqs / (CONST_K_B * T_K)) - 1)

@jit(nopython=True, fastmath=True)
def B_wav_nb(wavlens_cm: npt.NDArray[float], T_K: float) -> npt.NDArray[float]:
    return 2 * CONST_H * CONST_C**2 / wavlens_cm**5 / (np.exp(CONST_H * CONST_C / (wavlens_cm * (CONST_K_B * T_K))) - 1)
    

@jit(nopython=True, fastmath=True, parallel=True)
def L_vu_nb(
    freqs_Hz : npt.NDArray[float],
    Ts_K     : npt.NDArray[float],
    Aeffjs_cm2: npt.NDArray[float],
) -> npt.NDArray[float]:
    
    L_vus = np.zeros_like(freqs_Hz)
    nused = len(Ts_K)
    #debug_fact = np.zeros(nused)
    for i in prange(nused):
        dL_vus = 4 * pi * B_vu_nb(freqs_Hz, Ts_K[i]) * Aeffjs_cm2[i]
        L_vus += dL_vus
        ## debug
        #dL_sig = 4 * pi * CONST_SIG * Ts_K[i]**4 / pi * Aeffjs_cm2[i]
        #dL_int = np.trapezoid(dL_vus, freqs_Hz)
        #debug_fact[i] = dL_int/dL_sig
        #print(dL_int/dL_sig, Ts_K[i], Aeffjs_cm2[i])
    return L_vus#, debug_fact


@jit(nopython=True, fastmath=True, parallel=True)
def L_wav_nb(
    wavlens_cm: npt.NDArray[float],
    Ts_K      : npt.NDArray[float],
    Aeffjs_cm2 : npt.NDArray[float],
) -> npt.NDArray[float]:
    
    L_wavs= np.zeros_like(wavlens_cm)
    nused = len(Ts_K)
    for i in prange(nused):
        L_wavs += 4 * pi * B_wav_nb(wavlens_cm, Ts_K[i]) * Aeffjs_cm2[i]
    return L_wavs

.

## Debug

.

In [13]:
do_debug = False

.

## Main

.


In [14]:
if __name__ == '__main__' and not do_debug:

    # init combined data
    comb = {}
    
    for job_nickname in job_nicknames: #['2md', ]:  #
        job_profile = JOB_PROFILES_DICT[job_nickname]
        job_name    = job_profile['job_name']
        file_indexes= job_profile['file_indexes']  #[17600,]  #
        params      = job_profile['params']
        eos_opacity = get_eos_opacity(ieos=10, params=params)    #EoS_MESA_opacity(params, settings)
        
        comb[job_nickname] = {
            xyzs: {
                'times': np.full(len(file_indexes), np.nan) * units.yr,
                'lums' : np.full(len(file_indexes), np.nan) * units.Lsun,
                'lums_err': np.full(len(file_indexes), np.nan) * units.Lsun,
                'lums_olim' : np.full(len(file_indexes), np.nan) * units.Lsun,
                'lums_err_olim': np.full(len(file_indexes), np.nan) * units.Lsun,
                'Teffs'  : np.full(len(file_indexes), np.nan) * units.K,
                'Aphs_xy': np.full(len(file_indexes), np.nan) * units.au**2,
                'Rphs_z' : np.full(len(file_indexes), np.nan) * units.au,
                # no of particles at the photosphere - lower bound (weighted average per pixel, weighted by lums contribution)
                # i.e. how resolved the photosphere is
                'N_res': np.full(len(file_indexes), -1) * units.dimensionless_unscaled,
                'Nphs_kcs': np.full(len(file_indexes), -1) * units.dimensionless_unscaled,
                'wavlens': wavlens,
                'L_wavs': np.full((len(file_indexes), len(wavlens)), np.nan) * (units.Lsun/units.angstrom),
                'L_wavs_olim': np.full((len(file_indexes), len(wavlens)), np.nan) * (units.Lsun/units.angstrom),
                'params': {'PHOTOSPHERE_TAU': PHOTOSPHERE_TAU, 'z_olim_kc': z_olim_kc, **params},
                '_meta_': {
                    'lums' : { 'Description': "Luminosity.", },
                    'lums_olim' : { 'Description': "Luminosity, but ignoring source function of particles outside the surface"
                                    "(defined by summed column kernel kcs < z_olim_kc.", },
                    'Aphs_xy': { 'Description': (
                        "Visible size of the simulated object (i.e. pixel * (area per pixel) if tau > PHOTOSPHERE_TAU)."), },
                    'Rphs_z': { 'Description': (
                        "Visible size of the simulated object (i.e. pixel * (area per pixel))."), },
                    'N_res': { 'Description': (
                            "no of particles contributed for each ray - lower bound"
                            "(weighted average per pixel, weighted by lums contribution per pixel)"), },
                    'Nphs_kcs': { 'Description': (
                            "no of particles above the photosphere"
                            "Summed column kernel divided by expected average column kernel per particle"), },
                },
            } for xyzs in xyzs_list
        }

            
        for ifile, file_index in enumerate(file_indexes):
            # init
    
            mpdf = mpdf_read(job_name, file_index, eos_opacity, reset_xyz_by='R1', use_Tscales=use_Tscales, verbose=verbose)
            mpdf.calc_sdf_params(['R1'])
            sdf  = mpdf.data['gas']
            # kernel = sdf.kernel
            # kernel_rad = float(kernel.get_radius())
            # col_kernel = kernel.get_column_kernel_func(samples=1000)
            srcfuncs = mpdf.const['sigma_sb'] * sdf['T']**4 / pi
            sdf['srcfunc'] = srcfuncs

            with mupl.hdf5_open(
                f"{interm_dir}{job_nickname}_{file_index:05d}.lcgen.{no_xy_txt}.hdf5",
                'a', metadata) as out_interm_grp1:
                #out_interm_grp1 = mupl.hdf5_subgroup(out_interm_file, f"{file_index:05d}", {})

                for xyzs in xyzs_list:
                    xyzs_names_list = [x for x in xyzs]
        
                    # record time used
                    python_time_start = now()
                    print(f"Start: {python_time_start.isoformat()}")
                    print(f"\tWorking on {job_nickname}_{file_index:05d}_{xyzs}...")
        
                    
                    # get rays
                    rays, areas, dXs = get_xy_grids_of_rays(
                        sdf, no_xy=no_xy, frac_contained=100.,
                        use_adaptive_grid=False, xyzs_names_list=xyzs_names_list)
                    ray_areas = areas
                    pts    = np.array(sdf[xyzs_names_list])
                    hs     = np.array(sdf[ 'h' ])    # npart-shaped array
                    
                    rays_u = (rays * mpdf.units['dist']).to(units.au)
                    areas_u = (areas * mpdf.units['dist']**2).to(units.au**2)
        
                    
                    # do integration without error estimation
                    srcfuncs = np.array(srcfuncs)
                    srcfuncs_err = None # ask to re-calc below
                    (
                        lum, lum_err, rads, jfact_used,
                        lum_olim, lum_err_olim, rads_olim, jfact_olim_used,
                        estis, pphns, pphzs, indes, contr, pts_order_used,
                    ) = integrate_along_rays_gridxy(
                        sdf, srcfuncs, srcfuncs_err, rays, ray_areas,
                        nsample_pp=nsample_pp, z_olim_kc=z_olim_kc, photosphere_tau=PHOTOSPHERE_TAU,
                        xyzs_names_list=xyzs_names_list, parallel=True, verbose=verbose,
                    )

                    
                    # record time used
                    python_time_ended = now()
                    python_time__used  = python_time_ended - python_time_start
                    print(f"In Progress: {python_time_ended.isoformat()}\nTime Used: {python_time__used}\n")

                    
                    lum     = set_as_quantity(lum,     mpdf.units['lum']).to(units.Lsun)
                    lum_err = set_as_quantity(lum_err, mpdf.units['lum']).to(units.Lsun)
                    lum_olim     = set_as_quantity(lum_olim,     mpdf.units['lum']).to(units.Lsun)
                    lum_err_olim = set_as_quantity(lum_err_olim, mpdf.units['lum']).to(units.Lsun)
                    print(f"\n{job_nickname}_{file_index:05d}_{xyzs}:\n\n\t{lum = :.3f}, {lum_err = :.3f}\n")
                    print(f"Time = {mpdf.get_time(unit=units.yr):.2f}\n")

                    
                    # SEDs
                    Ts      = set_as_quantity(sdf['T'].iloc[pts_order_used], mpdf.units['temp']).cgs
                    Aeffjs      = set_as_quantity(jfact_used,      mpdf.units['dist']**2).cgs
                    Aeffjs_olim = set_as_quantity(jfact_olim_used, mpdf.units['dist']**2).cgs
                    L_wavs      = L_wav_nb(wavlens.cgs.value, Ts.cgs.value, Aeffjs.cgs.value     ) * (units.erg/units.s/units.cm)
                    L_wavs_olim = L_wav_nb(wavlens.cgs.value, Ts.cgs.value, Aeffjs_olim.cgs.value) * (units.erg/units.s/units.cm)
                    L_wavs      = L_wavs.to(     units.Lsun / units.angstrom)
                    L_wavs_olim = L_wavs_olim.to(units.Lsun / units.angstrom)
                    
                    L_int = np.trapezoid(L_wavs, wavlens).to(units.Lsun)
                    L_int_olim = np.trapezoid(L_wavs_olim, wavlens).to(units.Lsun)
                    print(f"{L_int      = :.3f} (rel err {(L_int/lum-1.).to(units.percent):.3f})\n")
                    print(f"{L_int_olim = :.3f} (rel err {(L_int_olim/lum_olim-1.).to(units.percent):.3f})\n")
                    print()


                    rads      = (rads      * mpdf.units['sigma_sb'] * mpdf.units['temp']**4 / units.sr).cgs
                    rads_olim = (rads_olim * mpdf.units['sigma_sb'] * mpdf.units['temp']**4 / units.sr).cgs
                    indes *= units.dimensionless_unscaled
                    contr = 100 * contr * units.percent
                    lum1  = ((4 * pi * units.sr) * (rads * areas_u)).sum().to(units.solLum)
                    print(f"Lum    : {lum1    = :12.2f} +/- {lum_err     :12.2f} ({(lum_err / lum1).to(units.percent): 6.2f})")
                    print(f"Low lim: {lum_olim= :12.2f} +/- {lum_err_olim:12.2f} ({(lum_err_olim/lum_olim).to(units.percent): 6.2f})")
                    estis  = (estis * mpdf.units['sigma_sb'] * mpdf.units['temp']**4 / units.sr).cgs
                    lume = ((4 * pi * units.sr) * (estis * areas_u)).sum().to(units.solLum)
                    print(f"Old est: {lume    = :12.2f}" +
                          f"    (rel diff = {(2 * (lum1 - lume) / (lum1 + lume)).to(units.percent):+6.2f})")
                    print()


                    try:
                        mask = np.logical_and(~np.isnan(contr), contr.value)
                        N_res = np.average(
                            1. / contr[mask], weights=(rads*areas_u)[mask].value,
                        ).to(units.dimensionless_unscaled)
                    except ZeroDivisionError:
                        N_res = 0.

                    # Aeffis= pones * areas_u
                    # area  = Aeffis.sum()
                    Zphs  = (pphzs * mpdf.units['dist']).to(units.au)
                    iphs = ~np.isnan(Zphs)
                    Nphs_kcs = pphns[iphs] * units.dimensionless_unscaled
                    Nph_kcs = np.average(Nphs_kcs)
                    Aph_xy= (iphs * areas_u).sum()
                    Teffs = ((rads * pi * units.sr / const.sigma_sb) ** 0.25).cgs
                    Teffs_olim = ((rads_olim * pi * units.sr / const.sigma_sb) ** 0.25).cgs
                    Teff = np.average(Teffs[iphs])
                    Rph_xy = (Aph_xy/pi)**0.5
                    Rphs_z = np.sqrt(Zphs**2 + np.sum(rays_u[:, 0, :2]**2, axis=1))
                    Rph_z = np.average(Rphs_z[iphs])
                    
    
                    if is_verbose(verbose, 'info'):
                        say('info', 'main()', verbose,
                            f"lum = {lum}",
                            # f"area (from <1>) = ({area  /areas_u.sum()*100: 5.1f}%) {area}",
                            f"photosphere T = {Teff:.3f} +/- {np.std(Teffs[iphs]):.3f}",
                            f"photosphere area = ({Aph_xy/areas_u.sum()*100: 5.1f}%) {Aph_xy:.3f}  (radius {Rph_xy:.3f})",
                            f"photosphere radius (from z) = {Rph_z:.3f} +/- {np.std(Rphs_z[iphs]):.3f}",
                            f"total possible area = {areas_u.sum():.3f}",
                            f"lower bound of the # of particles for each ray, weighted avg over lum per pixels = {N_res:.3f} ",
                            f"avg particles above photosphere (based on summed column kernel) per pixels = {Nph_kcs:.3f} ",
                        )
                
                    # save interm data
                    data = {}
                    data['lum'  ] = lum
                    data['lum_err' ] = lum_err
                    data['lum_olim'  ] = lum_olim
                    data['lum_err_olim' ] = lum_err_olim
                    data['Rph_z'] = Rph_z
                    data['Rph_xy'] = Rph_xy
                    data['Aph_xy'] = Aph_xy
                    data['Teff'] = Teff
                    data['N_res'] = N_res
                    data['Nph_kcs'] = Nph_kcs
                    data['Zphs'] = Zphs
                    data['Teffs'] = Teffs
                    data['Nphs_kcs'] = Nphs_kcs
                    data['xyzs' ] = xyzs
                    data['time' ] = mpdf.get_time()
                    data['mpdf_params'] = mpdf.params
                    data['rays' ] = rays_u[:, 0, :2]
                    data['ray_unit_vec'] = get_ray_unit_vec(rays_u[0].value)
                    data['area_per_ray'] = areas_u[0] #areas_u
                    data['rads' ] = rads
                    data['rads_olim' ] = rads_olim
                    data['contr'] = contr
                    data['wavlens'] = wavlens
                    data['L_wavs'] = L_wavs
                    data['L_wavs_olim'] = L_wavs_olim
                    # data['Aeffis'] = Aeffis
                    data['Aeffjs'] = Aeffjs
                    data['Aeffjs_olim'] = Aeffjs_olim
                    
                    data['_meta_'] = {
                        'N_res': comb[job_nickname][xyzs]['_meta_']['N_res'],
                        'rays' : { 'Description': "Pixel centers on the 2D plane defined by xyzs.", },
                        'rads' : { 'Description': "Specific intensity per pixel.", },
                        'contr': {
                            'Description': "Maximum contributed particle's contribution towards the specific intensity, per pixel.", },
                    }
    
                    
                    mupl.hdf5_dump(data, mupl.hdf5_subgroup(out_interm_grp1, xyzs, overwrite=True), {})
        
                    comb[job_nickname][xyzs]['times'][ifile] = data['time']
                    comb[job_nickname][xyzs]['lums' ][ifile] = data['lum' ]
                    comb[job_nickname][xyzs]['lums_err'][ifile] = data['lum_err']
                    comb[job_nickname][xyzs]['lums_olim' ][ifile] = data['lum_olim' ]
                    comb[job_nickname][xyzs]['lums_err_olim'][ifile] = data['lum_err_olim']
                    comb[job_nickname][xyzs]['Teffs'][ifile] = data['Teff']
                    comb[job_nickname][xyzs]['Aphs_xy'][ifile] = data['Aph_xy']
                    comb[job_nickname][xyzs]['Rphs_z'][ifile] = data['Rph_z']
                    comb[job_nickname][xyzs]['N_res'][ifile] = data['N_res']
                    comb[job_nickname][xyzs]['Nphs_kcs'][ifile] = data['Nph_kcs']
                    comb[job_nickname][xyzs]['L_wavs'][ifile] = data['L_wavs']
                    comb[job_nickname][xyzs]['L_wavs_olim'][ifile] = data['L_wavs_olim']

                    # debug
                    print()
                    print(f"{int(rads.size/2)-1 = }\n{rads[int(rads.size/2)-1].cgs = }")
                    inds = ~np.isnan(pphzs)
                    print(f"{np.count_nonzero(inds) / pphzs.size * 100} % rays hit photosphere")
                    print(f"{np.std(rads[inds]) / np.average(rads[inds]) = }")
                    lum_in_ph = (4*pi*units.sr*(rads[inds] * areas_u[inds]).sum()).to(units.Lsun)
                    print(f"{lum       = :.2f}\n{lum_in_ph = :.2f}    ({(lum_in_ph / lum).to(units.percent):.2f})")
        
                    # record time used
                    python_time_ended = now()
                    python_time__used  = python_time_ended - python_time_start
                    print(f"Ended: {python_time_ended.isoformat()}\nTime Used: {python_time__used}\n")


            # save data for now
            mupl.hdf5_dump({job_nickname: comb[job_nickname]}, f"{interm_dir}lcgen.{no_xy_txt}.{job_nickname}.tmp.hdf5", metadata)
        mupl.hdf5_dump({job_nickname: comb[job_nickname]}, f"{interm_dir}lcgen.{no_xy_txt}.{job_nickname}.hdf5", metadata)
                
    plt.close('all')
    mupl.hdf5_dump(comb, f"{interm_dir}lcgen.{no_xy_txt}.hdf5.gz", metadata)


*   Note   :    <module>() ==> mpdf_read() ==> read():
	

	Reading filename='../raw/clmu_2mdd/lumo_00000'


    Debug  :    <module>() ==> mpdf_read() ==> read():
	udist = 1.00036 solRad
	umass = 1.00035 solMass
	utime = 1.00053 unit_time
	self.time = np.float64(0.0)
self.gamma = np.float64(1.6666666666666667)
self.ieos = 10
self.total_mass = np.float64(1.6920736802719674)

	Center of mass location: self.loc_CoM = array([ 0.00323244, -0.00974289, -0.00362086])

*   Note   :    mpdf_read() ==> read() ==> reset_xyz_by():
	Reseting Origin to R1 ([0. 0. 0.])...
*   Note   :    mpdf_read() ==> read() ==> reset_xyz_by():
	CoM location is now [ 0.00323244 -0.00974289 -0.00362086]
**  Warning:    <module>() ==> mpdf_read() ==> read():
	kappa column exists.
	We here assume kappa is in phantom units self.units['opacity']=Unit("udist2 / umass") 
	However in phantom kappa is assumed to be in cgs unit.
	If so, please CONVERT KAPPA MANNUALLY into PHANTOM units BEFORE proceeding, e.g.:
		mpdf.data['g

FileNotFoundError: [Errno 2] No such file or directory: '../raw/clmu_2mdd/lumo_00010'